In [87]:
import nltk
import pandas as pd

writeups = pd.read_csv('./writeups.csv', header=0)


In [88]:
rows = []
for text in writeups["Text"]:
    rows.append({"text": text})
df = pd.DataFrame(rows)
print(len(df))


147


In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df['text'])
tfidf = vectorizer.transform(df['text'])
vocab = vectorizer.get_feature_names()

/Users/marco/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [90]:
from corextopic import corextopic as ct

anchors = []
model = ct.Corex(n_hidden=3, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

TypeError: unhashable type: 'list'

In [82]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: is, function, so, in, on, there, not, as, since, or
Topic #2: from, then, in the, from the, now, will, need, and the, be, need to
Topic #3: that, for, of, that the, of the, our, which is, after, source, with


In [83]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ['appears', 'user', 'visiting', 'visit', 'says', 'admin', 'link', 'form', 'following', 'login', 'code', 'page', 'website', 'challenge'],
    ['inspecting', 'getting', 'looked', 'making', 'entry', 'elements', 'output', 'override', 'reading', 'clear', 'const', 'help', 'tells', 'argument', 'unfortunately', 'users', 'logic', 'long', 'valid', 'exploit', 'controlled', 'leak', 'element', 'note', 'vulnerable', 'access', 'decode', 'statement', 'creates', 'load', 'provided', 'burpsuite', 'idea', 'calls', 'case', 'submit', 'assume', 'checks', 'pretty', 'condition', 'console', 'called', 'problem', 'similar', 'magic', 'different', 'random', 'think', 'client', 'response', 'site', 'provide', 'solve', 'result', 'shows', 'write', 'files', 'given', 'rendered', 'requests', 'happens', 'parameter', 'stored', 'read', 'variable', 'contains', 'web', 'want', 'work', 'gave', 'pass', 'thing', 'process', 'field', 'passed', 'interesting', 'browser', 'run', 'works', 'perform', 'right', 'html', 'text', 'directly', 'character', 'login', 'values', 'method', 'finally', 'way', 'injection', 'make', 'understand', 'post', 'bit', 'add', 'script', 'application', 'know', 'notice', 'create', 'css', 'operator', 'change', 'secret', 'query', 'includes', 'payload', 'path', 'endpoint', 'object', 'template', 'snippet', 'check', 'filter', 'time', 'inject', 'try', 'line', 'username', 'key', 'bypass', 'characters', 'content', 'look', 'data', 'means', 'need', 'set', 'challenge', 'account', 'request', 'type', 'following', 'password', 'index', 'js', 'string', 'like', 'javascript', 'using', 'used', 'website', 'server', 'php', 'array', 'looking', 'input', 'url', 'point', 'error', 'user', 'cookie', 'page', 'use', 'source', 'flag', 'file', 'function', 'value', 'admin', 'code'],
    ['data', 'input', 'method', 'local', 'decode', 'js', 'finally', 'force', 'set', 'function', 'execute', 'javascript', 'got', 'line', 'python', 'brute', 'add', 'path', 'query', 'list', 'session', 'inject', 'website', 'html', 'check', 'final', 'create', 'php', 'challenge', 'change', 'parameter', 'filter', 'look', 'result', 'object', 'injection', 'bypass', 'username', 'source', 'cookie', 'access', 'gives', 'http', 'try', 'error', 'array', 'server', 'tried', 'login', 'user', 'code', 'page', 'string', 'receive', 'script', 'password', 'use', 'file', 'following', 'admin', 'url', 'request', 'payload', 'flag']
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=3, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [84]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: page, following, the following, admin, on, get, on the, login, get the, code
Topic #2: need, function, server, source, user, code, script, value, in the, use
Topic #3: function, code, source, line, request, input, script, is, url, inject


In [85]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(3)]
).astype(float)
topic_df.index = df.index
df = pd.concat([df, topic_df], axis=1)

In [86]:
df.sample(5, random_state=42)

,text,topic_1,topic_2,topic_3
125,The code is nearly 300 lines... There is a log...,1.0,1.0,1.0
51,"So, my plan is simple. I will create an accoun...",1.0,1.0,1.0
138,Opening the website greets us with a 'login' p...,1.0,0.0,0.0
19,"We know that the username field must be ""admin...",0.0,1.0,1.0
104,"When we visit the website, we can see an input...",0.0,0.0,0.0
